In [1]:
from platform import python_version
python_version()

'3.10.6'

In [2]:
import os, glob, re, pickle
from functools import partial
from collections import OrderedDict
import operator as op
from cytoolz import compose

import pandas as pd
import seaborn as sns
import numpy as np
import scanpy as sc
import anndata as ad
import matplotlib as mpl
import matplotlib.pyplot as plt

from pyscenic.export import export2loom, add_scenic_metadata
from pyscenic.utils import load_motifs
from pyscenic.transform import df2regulons
from pyscenic.aucell import aucell
from pyscenic.binarization import binarize
from pyscenic.rss import regulon_specificity_scores
from pyscenic.plotting import plot_binarization, plot_rss

from IPython.display import HTML, display
import loompy as lp

In [3]:
# Set maximum number of jobs for Scanpy.
sc.settings.njobs = 32

In [26]:
os.getcwd()

'/mnt/cho_lab/disk2/jiayuzh/projects/perianal-cd/analysis/scenic'

In [4]:
os.chdir('/mnt/cho_lab/disk2/jiayuzh/projects/perianal-cd/analysis/scenic')

Folder Structure

In [5]:
RESOURCES_FOLDERNAME = "./resources/"
AUXILLIARIES_FOLDERNAME = "./auxilliaries/"
RESULTS_FOLDERNAME = "./results/"
FIGURES_FOLDERNAME = "./figures/"

In [6]:
sc.settings.figdir = FIGURES_FOLDERNAME

Auxilliary functions.

In [7]:
BASE_URL = "http://motifcollections.aertslab.org/v10/logos/"
COLUMN_NAME_LOGO = "MotifLogo"
COLUMN_NAME_MOTIF_ID = "MotifID"
COLUMN_NAME_TARGETS = "TargetGenes"

In [8]:
def savesvg(fname: str, fig, folder: str=FIGURES_FOLDERNAME) -> None:
    """
    Save figure as vector-based SVG image format.
    """
    fig.tight_layout()
    fig.savefig(os.path.join(folder, fname), format='svg')

In [9]:
def display_logos(df: pd.DataFrame, top_target_genes: int = 3, base_url: str = BASE_URL):
    """
    :param df:
    :param base_url:
    """
    # Make sure the original dataframe is not altered.
    df = df.copy()
    
    # Add column with URLs to sequence logo.
    def create_url(motif_id):
        return '<img src="{}{}.png" style="max-height:124px;"></img>'.format(base_url, motif_id)
    df[("Enrichment", COLUMN_NAME_LOGO)] = list(map(create_url, df.index.get_level_values(COLUMN_NAME_MOTIF_ID)))
    
    # Truncate TargetGenes.
    def truncate(col_val):
        return sorted(col_val, key=op.itemgetter(1))[:top_target_genes]
    df[("Enrichment", COLUMN_NAME_TARGETS)] = list(map(truncate, df[("Enrichment", COLUMN_NAME_TARGETS)]))
    
    MAX_COL_WIDTH = pd.get_option('display.max_colwidth')
    pd.set_option('display.max_colwidth', -1)
    display(HTML(df.head().to_html(escape=False)))
    pd.set_option('display.max_colwidth', MAX_COL_WIDTH)

Auxilliary data sets.

In [10]:
# Downloaded fromm pySCENIC github repo: https://github.com/aertslab/pySCENIC/tree/master/resources
HUMAN_TFS_FNAME = os.path.join(AUXILLIARIES_FOLDERNAME, 'allTFs_hg38.txt')
# Ranking databases. Downloaded from cisTargetDB: https://resources.aertslab.org/cistarget/
RANKING_DBS_FNAMES = list(map(lambda fn: os.path.join(AUXILLIARIES_FOLDERNAME, fn),
                       ['hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather',
                       'hg38_500bp_up_100bp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather']))
# Motif annotations. Downloaded from cisTargetDB: https://resources.aertslab.org/cistarget/
MOTIF_ANNOTATIONS_FNAME = os.path.join(AUXILLIARIES_FOLDERNAME, 'motifs-v10nr_clust-nr.hgnc-m0.001-o0.0.tbl')

Perianal CD Fistula

In [11]:
DATASET_ID = "Multi"
PROCESS_CODE = "filtered"

Reading in expression data: 10x Genomics mtx files

In [12]:
adata = sc.read_10x_mtx(
    './resources/Perianal_multi/' ,                 # the directory with the `.mtx` file
    var_names='gene_symbols') 

result created

In [13]:
METADATA_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.metadata.csv'.format(DATASET_ID))
EXP_MTX_QC_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.qc.tpm.csv'.format(DATASET_ID))
ADJACENCIES_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.adjacencies.tsv'.format(DATASET_ID))
MOTIFS_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.motifs.csv'.format(DATASET_ID))
REGULONS_DAT_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.regulons.dat'.format(DATASET_ID))
AUCELL_MTX_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.auc.csv'.format(DATASET_ID))
BIN_MTX_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.bin.csv'.format(DATASET_ID))
THR_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.thresholds.csv'.format(DATASET_ID))
ANNDATA_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.h5ad'.format(DATASET_ID))
LOOM_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}_{}.loom'.format(DATASET_ID, PROCESS_CODE))


In [14]:
nCountsPerGene = np.sum(adata.X, axis=0)
nCellsPerGene = np.sum(adata.X>0, axis=0)

# Show info
print("Number of counts (in the dataset units) per gene:", nCountsPerGene.min(), " - " ,nCountsPerGene.max())
print("Number of cells in which each gene is detected:", nCellsPerGene.min(), " - " ,nCellsPerGene.max())

Number of counts (in the dataset units) per gene: 0.0  -  88752216.0
Number of cells in which each gene is detected: 0  -  48744


STEP 0: Preprocessing

In [15]:
df_metadata = pd.read_table('./resources/meta_multi.txt')
df_metadata.head()

,cell_id,sample_id,cell_type,cohort,no.of.genes,no.of.reads,patient_id,age,sex,tnf_treatment,fistula_current,inflammation,site
0,AAACCCAAGAAGGATG-1_2_1_1_1_1_1_1_1_1_1_1_1_1_1...,Perianal5_Rect,IgA Plasma,White,1305,15351,Perianal5,23,Male,No,Yes,Non_inflamed,Rectum
1,AAACCCAAGGGAGTGG-1_2_1_1_1_1_1_1_1_1_1_1_1_1_1...,Perianal5_Rect,IgA Plasma,White,608,4030,Perianal5,23,Male,No,Yes,Non_inflamed,Rectum
2,AAACCCAAGTTGGACG-1_2_1_1_1_1_1_1_1_1_1_1_1_1_1...,Perianal5_Rect,CD4 T,White,302,753,Perianal5,23,Male,No,Yes,Non_inflamed,Rectum
3,AAACCCACAGAGGTAC-1_2_1_1_1_1_1_1_1_1_1_1_1_1_1...,Perianal5_Rect,IgG Plasma,White,1301,11817,Perianal5,23,Male,No,Yes,Non_inflamed,Rectum
4,AAACGAAAGTCCCAAT-1_2_1_1_1_1_1_1_1_1_1_1_1_1_1...,Perianal5_Rect,IgG Plasma,White,916,6442,Perianal5,23,Male,No,Yes,Non_inflamed,Rectum


EXPRESSION MATRIX QC

In [16]:
df_obs = df_metadata.set_index('cell_id')
adata.obs = df_obs
adata.var_names_make_unique()

In [17]:
# simply compute the number of genes per cell (computers 'n_genes' column)
sc.pp.filter_cells(adata, min_genes=0)
# mito and genes/counts cuts
mito_genes = adata.var_names.str.startswith('MT-')
# for each cell compute fraction of counts in mito genes vs. all genes
adata.obs['percent_mito'] = np.sum(
    adata[:, mito_genes].X, axis=1).A1 / np.sum(adata.X, axis=1).A1
# add the total counts per cell as observations-annotation to adata
adata.obs['n_counts'] = adata.X.sum(axis=1).A1

In [18]:
# initial cuts
sc.pp.filter_cells(adata, min_genes=200 )
sc.pp.filter_genes(adata, min_cells=3 )
adata = adata[adata.obs['n_genes'] < 4000, :]
adata = adata[adata.obs['percent_mito'] < 0.15, :]

In [19]:
adata.raw = adata 
sc.pp.log1p(adata)
adata

AnnData object with n_obs × n_vars = 42168 × 22897
    obs: 'sample_id', 'cell_type', 'cohort', 'no.of.genes', 'no.of.reads', 'patient_id', 'age', 'sex', 'tnf_treatment', 'fistula_current', 'inflammation', 'site', 'n_genes', 'percent_mito', 'n_counts'
    var: 'gene_ids', 'feature_types', 'n_cells'
    uns: 'log1p'

In [42]:
adata.write_h5ad(ANNDATA_FNAME)

In [43]:
# create basic row and column attributes for the loom file:
row_attrs = {
    "Gene": np.array(adata.var_names) ,
}
col_attrs = {
    "CellID": np.array(adata.obs_names) ,
    "nGene": np.array( np.sum(adata.X.transpose()>0 , axis=0)).flatten() ,
    "nUMI": np.array( np.sum(adata.X.transpose() , axis=0)).flatten() ,
}
lp.create(LOOM_FNAME, adata.X.transpose(), row_attrs, col_attrs)